In [7]:
import pandas as pd
import numpy as np
from glob import glob
import pylab as plt
import seaborn as sns
%matplotlib inline

In [8]:
teams = ['Columbia_UNC-SurvCon', 'QJHong-Encounter']

In [33]:
for target_type in ['case', 'death']: 
    print(f'\n=========== {target_type} ===========')
    otherTeam_fname = f'../otherTeam/data_processed/otherTeam_{target_type}.csv'
    df_otherTeams = pd.read_csv(otherTeam_fname, index_col=0)
    # df_otherTeams = df_otherTeams.rename(columns={col: col.split('-')[0] for col in df_otherTeams.columns})
    # print(df_otherTeams)
    
    ours_fname = f'results_time/methodCompare_{target_type}_ours.csv'
    df_ours = pd.read_csv(ours_fname, index_col=0)[['gnd', 'prd-0_1']].rename(columns={'prd-0_1': 'UnIT'})
    # print(df_ours)
    
    df_combined = df_ours.join(df_otherTeams, how='outer')# .dropna().
    df_combined = df_combined.loc['2020-07-18':]
    # print(df_combined)
    
    df_err = pd.DataFrame(index=df_combined.index)
    for col in df_combined.columns:
        if col == 'gnd':
            continue
        df_err[col] = df_combined[col] - df_combined['gnd']
    mabs = df_err.abs().mean().sort_values(ascending=False)#[-10:]
    df_mabs = pd.DataFrame({'team': mabs.index, 'err': mabs.values})
    df_mabs.to_csv(f'results_time/compare_with_otherTeams_{target_type}_err_raw.csv', index=False, float_format='%.0f')
    # rmse = df_err.pow(2).mean().pow(.5).sort_values()

    print(mabs)
    # print(rmse)

    df_combined = pd.concat([df_otherTeams, df_ours], axis=1, join='outer').round(0)
    df_combined.to_csv(f'results_time/compare_with_otherTeams_{target_type}.csv', float_format='%.0f')


=========== case ===========
CovidAnalytics-DELPHI    174477.600000
JHU_IDD-CovidSP          138125.684211
IowaStateLW-STEM          92378.000000
Covid19Sim-Simulator      91198.611111
RobertWalraven-ESG        87452.263158
LANL-GrowthRate           79009.000000
UCLA-SuEIR                72387.150000
CU-nochange               68956.588235
COVIDhub-baseline         65962.700000
Columbia_UNC-SurvCon      64173.500000
COVIDhub-ensemble         62720.000000
USC-SI_kJalpha            62424.600000
UMich-RidgeTfReg          61521.150000
OliverWyman-Navigator     61183.100000
Geneva-DetGrowth          60886.100000
QJHong-Encounter          57657.100000
LNQ-ens1                  55341.950000
UnIT                      39847.150000
dtype: float64

=========== death ===========
UMich-RidgeTfReg         1487.500000
RobertWalraven-ESG       1457.421053
LANL-GrowthRate          1093.800000
JHU_IDD-CovidSP          1029.100000
Covid19Sim-Simulator      877.333333
IowaStateLW-STEM          845.105263


/home/yihuang/.local/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [23]:
# method = 0
# for target_type in ['case', 'death']:
#     df_combined = pd.read_csv(f'results_time/compare_with_otherTeams_{target_type}.csv', index_col=0)

#     fig, axes = plt.subplots(1, 2, figsize=(14, 5))
#     cols = ['Columbia_1', 'QJHong_1', f'prd-{method}_1', 'gnd']

#     df_tmp = df_combined.dropna()

#     for col in cols:
#         if col == 'gnd':
#             continue
#         err = (df_tmp[col] - df_tmp['gnd']).values
#         rmse = np.sqrt((err ** 2).mean())
#         mabs = np.abs(err).mean()
#         sns.distplot(err, ax=axes[0], label=f'{col}\nmabs={mabs:.2e}')
#         sns.distplot(err, ax=axes[1], label=f'{col}\nrmse={rmse:.2e}')
#         axes[0].legend()
#         axes[1].legend()
#         axes[0].set_title(f'{target_type} mean absolute error', fontsize=15)
#         axes[1].set_title(f'{target_type} mean squared error', fontsize=15)

In [17]:
# cols = ['Columbia_1', 'QJHong_1', f'prd-{method}_1']

# for target_type in ['case', 'death']:
    
#     prd_fname = f'results_time/compare_with_otherTeams_{target_type}.csv'
#     err_fname = f'results_time/compare_with_otherTeams_{target_type}_err.csv'
    
#     df_combined = pd.read_csv(prd_fname, index_col=0).dropna()
    
#     df_err = pd.DataFrame(index=df_combined.index)
#     for col in cols:
#         df_err[col] = df_combined[col] - df_combined['gnd']
    
#     print(df_err)
#     df_err.to_csv(err_fname, header=False, index=False, sep=' ', float_format='%.0f')

In [52]:
err_dfs = [] 
for target_type in ['case', 'death']:
    tmp = pd.read_csv(f'results_time/compare_with_otherTeams_{target_type}_err_raw.csv', index_col=0)
    tmp.rename(columns={'err': f'err_{target_type}'}, inplace=True)
    err_dfs.append(tmp)
    
df_err = pd.concat(err_dfs, axis=1,join='inner')

In [55]:
func = lambda x: x.split('-')[0]
df_err_case = df_err['err_case'].sort_values(ascending=False)[-15:].copy()
df_err_case.rename(index={ind: func(ind) for ind in df_err_case.index}, inplace=True)
df_err_death = df_err['err_death'].sort_values(ascending=False)[-15:].copy()
df_err_death.rename(index={ind: func(ind) for ind in df_err_death.index}, inplace=True)
df_err_case.to_csv(f'results_time/compare_with_otherTeams_case_err_raw.csv')
df_err_death.to_csv(f'results_time/compare_with_otherTeams_death_err_raw.csv')

/home/yihuang/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
/home/yihuang/.local/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  import sys
